In [1]:
import firebase_admin
from firebase_admin import credentials, firestore
from PIL import Image
import requests
from io import BytesIO
import os
from urllib.parse import urlparse

# Initialize Firebase
cred1 = credentials.Certificate("C:/Users/aishi/OneDrive/Desktop/FR/snappy.json")
firebase_admin.initialize_app(cred1)
db = firestore.client()

def display_image_from_url(image_url, save_folder):
    response = requests.get(image_url)
    if response.status_code == 200:
        image_data = response.content
        img = Image.open(BytesIO(image_data))
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        filename = os.path.basename(urlparse(image_url).path)
        save_path = os.path.join(save_folder, filename)
        img.save(save_path)
        print(f"Image saved to: {save_path}")
        img.show()
    else:
        print("Failed to fetch image from URL:", image_url)

def get_all_docs(collection_name, field_name, save_folder):
    docs = db.collection(collection_name).stream()
    for doc in docs:
        doc_data = doc.to_dict()
        if field_name in doc_data:
            image_url = doc_data[field_name]
            display_image_from_url(image_url, save_folder)

def get_docs_with_status(coll_name, s_value, field_name, save_folder):
    query = db.collection(coll_name).where("Status", "==", s_value).stream()
    for doc in query:
        doc_data = doc.to_dict()
        if field_name in doc_data:
            image_url = doc_data[field_name]
            display_image_from_url(image_url, save_folder)

# Example usage for "users" collection
field_name = 'registration_photo'
save_folder = "swastik"
get_all_docs('users', field_name, save_folder)
get_docs_with_status('users', 'some_value', field_name, save_folder)


Image saved to: swastik\users%2FYlOLdp44L8eombjQAboXHdtXaEf1%2Fuploads%2F1706874833328000.jpg
Image saved to: swastik\users%2FCgEAkvaWjOURUSePaz87Bxqf3Uy2%2Fuploads%2F1706884176430000.jpg
Image saved to: swastik\users%2FwACIvxif92gvA3FLzJoOdIiW2Ay2%2Fuploads%2F1706871995429000.jpg
Image saved to: swastik\users%2FLp9g7VwPugThNNiuWlKuv1Q9IFV2%2Fuploads%2F1706884748264000.png


c:\Users\aishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:295: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
